In [1]:

import pandas as pd
import numpy as np
import os
from collections import defaultdict
from datetime import datetime

# ----------------------------------------------------------------------
# FUNCIONES
# ----------------------------------------------------------------------

def weighted_average(df, value_col, weight_col):
    """Calcula el promedio ponderado de una columna usando pesos (factores de expansion)."""
    df_filtered = df.dropna(subset=[value_col, weight_col])

    # Excluir valores de ingreso no validos o cero para promedios de ingreso
    if value_col in ['ingocup', 'ing_x_hrs', 'anios_esc', 'hrsocup']:
        # Para anos de escolaridad y horas, aceptamos 0, pero filtramos nulos reales
        if value_col in ['ingocup', 'ing_x_hrs']:
            df_filtered = df_filtered[df_filtered[value_col] > 0].copy()
        else:
            # Para horas y educacion, filtramos valores no especificados (99) si existen
            df_filtered = df_filtered[df_filtered[value_col] < 99].copy()

    if df_filtered.empty or df_filtered[weight_col].sum() == 0:
        return np.nan

    return np.average(df_filtered[value_col], weights=df_filtered[weight_col])


def safe_pct(num, den):
    """Regresa porcentaje con control de division entre cero."""
    return (num / den) * 100 if den else np.nan


# Diccionario de Entidades
ENTIDADES = {
    1: 'Aguascalientes', 2: 'Baja California', 3: 'Baja California Sur', 4: 'Campeche',
    5: 'Coahuila', 6: 'Colima', 7: 'Chiapas', 8: 'Chihuahua', 9: 'Ciudad de Mexico',
    10: 'Durango', 11: 'Guanajuato', 12: 'Guerrero', 13: 'Hidalgo', 14: 'Jalisco',
    15: 'Mexico', 16: 'Michoacan', 17: 'Morelos', 18: 'Nayarit', 19: 'Nuevo Leon',
    20: 'Oaxaca', 21: 'Puebla', 22: 'Queretaro', 23: 'Quintana Roo', 24: 'San Luis Potosi',
    25: 'Sinaloa', 26: 'Sonora', 27: 'Tabasco', 28: 'Tamaulipas', 29: 'Tlaxcala',
    30: 'Veracruz', 31: 'Yucatan', 32: 'Zacatecas'
}


def obtener_nombre_archivo(year, quarter, file_format='dta'):
    #"""Determina el nombre del archivo SDEMT segun el periodo y la nueva ruta."""
    year_short = str(year)[-2:]
    base_name = None

    # Logica de Nomenclatura
    if year <= 2018:
        base_name = f"SDEMT{quarter}{year_short}".upper()
    elif year == 2019:
        base_name = f"sdemt{quarter}{year_short}".lower()
    elif (year == 2020 and quarter >= 3) or year in [2021, 2022]:
        base_name = f"ENOEN_SDEMT{quarter}{year_short}".upper()
    elif year >= 2023:
        base_name = f"ENOE_SDEMT{quarter}{year_short}".upper()

    dir_name = f"ENOE_{year}_{quarter}"
    file_name = f"{base_name}.{file_format}" if base_name else None
    file_path = os.path.join("Data/ENOE_dta", dir_name, file_name) if file_name else None
    return file_path


def pedir_rango_trimestral():
    #"""Pide al usuario el rango de anos y trimestres."""
    while True:
        try:
            print("\n--- Definicion del Rango de la Serie de Tiempo ---")
            start_year = int(input("Ingrese el ANO de inicio (e.g., 2005): "))
            start_quarter = int(input("Ingrese el TRIMESTRE de inicio (1 a 4): "))
            end_year = int(input("Ingrese el ANO final (e.g., 2025): "))
            end_quarter = int(input("Ingrese el TRIMESTRE final (1 a 4): "))

            if not (1 <= start_quarter <= 4 and 1 <= end_quarter <= 4):
                raise ValueError("El trimestre debe ser un numero entre 1 y 4.")
            start_date = datetime(start_year, start_quarter * 3 - 2, 1)
            end_date = datetime(end_year, end_quarter * 3 - 2, 1)
            if start_date > end_date:
                raise ValueError("El periodo de inicio debe ser anterior al final.")
            break
        except ValueError as e:
            print(f"Entrada invalida: {e}.")

    periodos = []
    current_year = start_year
    current_quarter = start_quarter
    while current_year < end_year or (current_year == end_year and current_quarter <= end_quarter):
        if current_year == 2020 and current_quarter in [1, 2]:
            print(f"--- Aviso: Saltando periodo {current_year} T{current_quarter} (No disponible). ---")
            periodos.append((current_year, current_quarter))
        else:
            periodos.append((current_year, current_quarter))

        if current_quarter == 4:
            current_quarter = 1
            current_year += 1
        else:
            current_quarter += 1
    return periodos


# ----------------------------------------------------------------------
# FUNCION PRINCIPAL DE PROCESAMIENTO
# ----------------------------------------------------------------------

def procesar_trimestre_enoe(year, quarter, file_format='dta'):
    periodo_str = f"{year} T{quarter}"
    print(f"\n--- Procesando: {periodo_str} ---")

    file_path = obtener_nombre_archivo(year, quarter, file_format)

    is_fac_tri_period = (
        (year == 2020 and quarter >= 3) or
        (year >= 2021)
    )

    PONDERATOR = 'fac_tri' if is_fac_tri_period else 'fac'

    if file_path is None or not os.path.exists(file_path):
        print(f"Error: Archivo no encontrado: {file_path}")
        return None, None

    try:
        df = pd.read_stata(file_path, convert_categoricals=False)
        if df.empty:
            return None, None
        print(f"Cargado: {len(df):,} regs. Ponderador: {PONDERATOR.upper()}")
    except Exception as e:
        print(f"Error lectura: {e}")
        return None, None

    # --- Limpieza ---
    df.columns = df.columns.str.lower()

    columnas_requeridas = [
        PONDERATOR, 'r_def', 'c_res', 'ent', 'sex', 'eda', 'clase1', 'clase2',
        'pos_ocu', 'emp_ppal', 'sub_o', 'ingocup', 'ing_x_hrs',
        'hrsocup', 'niv_ins', 'anios_esc', 'seg_soc'
    ]

    for col in columnas_requeridas:
        if col not in df.columns:
            if col == PONDERATOR:
                return None, None
            df[col] = 0 if col not in ['ingocup', 'ing_x_hrs'] else np.nan

    df['r_def'] = df['r_def'].astype(str).str.strip()
    cols_int = ['sex', 'eda', 'clase1', 'clase2', 'c_res', 'ent', 'pos_ocu',
                'emp_ppal', 'sub_o', 'niv_ins', 'seg_soc']
    for col in cols_int:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(-1).astype(int)

    cols_float = ['ingocup', 'ing_x_hrs', 'hrsocup', 'anios_esc', PONDERATOR]
    for col in cols_float:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # --- Filtros Base ---
    df_base = df[(df['r_def'] == '0.0') & (df['c_res'].isin([1, 3]))].copy()
    df_15_y_mas = df_base[df_base['eda'].between(15, 98)].copy()

    if df_15_y_mas.empty:
        return None, None

    df_base['ent_nombre'] = df_base['ent'].map(ENTIDADES)
    df_15_y_mas['ent_nombre'] = df_15_y_mas['ent'].map(ENTIDADES)

    df_pea = df_15_y_mas[df_15_y_mas['clase1'] == 1].copy()
    df_pnea = df_15_y_mas[df_15_y_mas['clase1'] == 2].copy()
    df_ocupada = df_15_y_mas[df_15_y_mas['clase2'] == 1].copy()

    df_ocupada_h = df_ocupada[df_ocupada['sex'] == 1]
    df_ocupada_m = df_ocupada[df_ocupada['sex'] == 2]
    df_15_h = df_15_y_mas[df_15_y_mas['sex'] == 1]
    df_15_m = df_15_y_mas[df_15_y_mas['sex'] == 2]
    df_ocup_ing = df_ocupada[df_ocupada['ingocup'] > 0].copy()
    df_ocup_ing_h = df_ocup_ing[df_ocup_ing['sex'] == 1]
    df_ocup_ing_m = df_ocup_ing[df_ocup_ing['sex'] == 2]

    # Poblaciones base para porcentajes
    pob_total = df_base[PONDERATOR].sum()
    pob_15_total = df_15_y_mas[PONDERATOR].sum()
    pob_15_h = df_15_h[PONDERATOR].sum()
    pob_15_m = df_15_m[PONDERATOR].sum()

    formal_total = df_ocupada[df_ocupada['emp_ppal'] == 2][PONDERATOR].sum()
    informal_total = df_ocupada[df_ocupada['emp_ppal'] == 1][PONDERATOR].sum()
    no_remunerados_total = df_ocupada[df_ocupada['pos_ocu'] == 4][PONDERATOR].sum()

    # ------------------------------------------------------------------
    # --- CALCULOS NACIONALES (VARIABLES EXPANDIDAS) ---
    # ------------------------------------------------------------------

    datos_nacional = {
        'year': year, 'quarter': quarter,

        # --- Basicos ---
        'pob_total': pob_total,
        'pob_15ymas_total': pob_15_total,
        'pob_15ymas_hombres': pob_15_h,
        'pob_15ymas_mujeres': pob_15_m,
        'pct_15ymas_sobre_total': safe_pct(pob_15_total, pob_total),
        'pea_total': df_pea[PONDERATOR].sum(),
        'ocupada_total': df_ocupada[PONDERATOR].sum(),
        'desocupada_total': df_15_y_mas[df_15_y_mas['clase2'] == 2][PONDERATOR].sum(),

        # --- Masa salarial (ingreso mensual ponderado) ---
        'masa_salarial_total': (df_ocup_ing['ingocup'] * df_ocup_ing[PONDERATOR]).sum(),
        'masa_salarial_hombres': (df_ocup_ing_h['ingocup'] * df_ocup_ing_h[PONDERATOR]).sum(),
        'masa_salarial_mujeres': (df_ocup_ing_m['ingocup'] * df_ocup_ing_m[PONDERATOR]).sum(),

        # --- 1. Ingreso Promedio por Hora (Hombre/Mujer) ---
        'ing_hora_hombres': weighted_average(df_ocupada_h, 'ing_x_hrs', PONDERATOR),
        'ing_hora_mujeres': weighted_average(df_ocupada_m, 'ing_x_hrs', PONDERATOR),

        # --- 2. Ingreso Promedio Mensual (Hombre/Mujer) ---
        'ing_mensual_hombres': weighted_average(df_ocupada_h, 'ingocup', PONDERATOR),
        'ing_mensual_mujeres': weighted_average(df_ocupada_m, 'ingocup', PONDERATOR),

        # --- 3. Horas Trabajadas Semanales Promedio (Hombre/Mujer) ---
        # Nota: hrsocup es semanal.
        'horas_sem_hombres': weighted_average(df_ocupada_h, 'hrsocup', PONDERATOR),
        'horas_sem_mujeres': weighted_average(df_ocupada_m, 'hrsocup', PONDERATOR),

        # --- 4. Ocupacion Formal (Hombre/Mujer) ---
        'formal_total': formal_total,
        'formal_hombres': df_ocupada_h[df_ocupada_h['emp_ppal'] == 2][PONDERATOR].sum(),
        'formal_mujeres': df_ocupada_m[df_ocupada_m['emp_ppal'] == 2][PONDERATOR].sum(),
        'informal_total': informal_total,
        'informal_hombres': df_ocupada_h[df_ocupada_h['emp_ppal'] == 1][PONDERATOR].sum(),
        'informal_mujeres': df_ocupada_m[df_ocupada_m['emp_ppal'] == 1][PONDERATOR].sum(),

        # --- 5. Trabajadores No Remunerados (Hombre/Mujer) ---
        'no_remunerados_total': no_remunerados_total,
        'no_remunerados_hombres': df_ocupada_h[df_ocupada_h['pos_ocu'] == 4][PONDERATOR].sum(),
        'no_remunerados_mujeres': df_ocupada_m[df_ocupada_m['pos_ocu'] == 4][PONDERATOR].sum(),

        # --- 7. Promedio Anios Escolaridad (Total, Hombre, Mujer) ---
        'anios_esc_total': weighted_average(df_15_y_mas, 'anios_esc', PONDERATOR),
        'anios_esc_hombres': weighted_average(df_15_h, 'anios_esc', PONDERATOR),
        'anios_esc_mujeres': weighted_average(df_15_m, 'anios_esc', PONDERATOR),

        # --- 8. Acceso a Atencion Medica ---
        'pob_con_salud': df_ocupada[df_ocupada['seg_soc'] == 1][PONDERATOR].sum(),
        'pob_sin_salud': df_ocupada[df_ocupada['seg_soc'] == 2][PONDERATOR].sum(),
    }

    datos_nacional['pct_formal_total_15ymas'] = safe_pct(formal_total, pob_15_total)
    datos_nacional['pct_formal_hombres_15ymas'] = safe_pct(datos_nacional['formal_hombres'], pob_15_h)
    datos_nacional['pct_formal_mujeres_15ymas'] = safe_pct(datos_nacional['formal_mujeres'], pob_15_m)

    datos_nacional['pct_informal_total_15ymas'] = safe_pct(informal_total, pob_15_total)
    datos_nacional['pct_informal_hombres_15ymas'] = safe_pct(datos_nacional['informal_hombres'], pob_15_h)
    datos_nacional['pct_informal_mujeres_15ymas'] = safe_pct(datos_nacional['informal_mujeres'], pob_15_m)

    datos_nacional['pct_no_remunerados_total_15ymas'] = safe_pct(no_remunerados_total, pob_15_total)
    datos_nacional['pct_no_remunerados_hombres_15ymas'] = safe_pct(datos_nacional['no_remunerados_hombres'], pob_15_h)
    datos_nacional['pct_no_remunerados_mujeres_15ymas'] = safe_pct(datos_nacional['no_remunerados_mujeres'], pob_15_m)

    datos_nacional['pct_con_salud'] = safe_pct(
        datos_nacional['pob_con_salud'],
        datos_nacional['pob_con_salud'] + datos_nacional['pob_sin_salud']
    )

    niveles_educ = {1: 'prim_inc', 2: 'prim_comp', 3: 'secundaria', 4: 'sup_y_mas'}

        # ------------------------------------------------------------------
    # NUEVOS CALCULOS (NACIONAL):
    # 1) Promedio de HORAS MENSUALES remuneradas por grado educativo y sexo
    # 2) Formalidad/Informalidad (%) por grado educativo y sexo
    # ------------------------------------------------------------------

    # hrsocup en ENOE suele ser HORAS SEMANALES. Para horas mensuales usamos 52/12 semanas por mes.
    WEEKS_PER_MONTH = 52 / 12  # ~= 4.3333

    # Definicion de "ocupación remunerada":
    # - Excluimos no remunerados (pos_ocu == 4)
    # - Requerimos ingocup > 0 (si prefieres incluir ingreso 0 pero ocupados, cambia este filtro)
    df_ocup_rem = df_ocupada[(df_ocupada['pos_ocu'] != 4) & (df_ocupada['ingocup'] > 0)].copy()

    # Subconjuntos por sexo (1=hombre, 2=mujer)
    df_ocup_rem_h = df_ocup_rem[df_ocup_rem['sex'] == 1]
    df_ocup_rem_m = df_ocup_rem[df_ocup_rem['sex'] == 2]

    for codigo, etiqueta in niveles_educ.items():
        # ---- Filtro por nivel educativo (ocupación remunerada) ----
        df_e = df_ocup_rem[df_ocup_rem['niv_ins'] == codigo]
        df_e_h = df_e[df_e['sex'] == 1]
        df_e_m = df_e[df_e['sex'] == 2]

        # ==========================================================
        # (1) HORAS MENSUALES PROMEDIO remuneradas (Total/H/M)
        # ==========================================================
        # Promedio semanal ponderado
        hrs_sem_total = weighted_average(df_e, 'hrsocup', PONDERATOR)
        hrs_sem_h = weighted_average(df_e_h, 'hrsocup', PONDERATOR)
        hrs_sem_m = weighted_average(df_e_m, 'hrsocup', PONDERATOR)

        # Convertimos a mensual
        datos_nacional[f'hrs_mens_rem_{etiqueta}_total'] = (
            hrs_sem_total * WEEKS_PER_MONTH if pd.notna(hrs_sem_total) else np.nan
        )
        datos_nacional[f'hrs_mens_rem_{etiqueta}_hombres'] = (
            hrs_sem_h * WEEKS_PER_MONTH if pd.notna(hrs_sem_h) else np.nan
        )
        datos_nacional[f'hrs_mens_rem_{etiqueta}_mujeres'] = (
            hrs_sem_m * WEEKS_PER_MONTH if pd.notna(hrs_sem_m) else np.nan
        )

        # ==========================================================
        # (2) FORMALIDAD / INFORMALIDAD (%) dentro de ese nivel educativo
        # ==========================================================
        # Denominadores (ocupación remunerada dentro del nivel)
        den_total = df_e[PONDERATOR].sum()
        den_h = df_e_h[PONDERATOR].sum()
        den_m = df_e_m[PONDERATOR].sum()

        # Numeradores formal/informal
        formal_total_e = df_e[df_e['emp_ppal'] == 2][PONDERATOR].sum()
        informal_total_e = df_e[df_e['emp_ppal'] == 1][PONDERATOR].sum()

        formal_h_e = df_e_h[df_e_h['emp_ppal'] == 2][PONDERATOR].sum()
        informal_h_e = df_e_h[df_e_h['emp_ppal'] == 1][PONDERATOR].sum()

        formal_m_e = df_e_m[df_e_m['emp_ppal'] == 2][PONDERATOR].sum()
        informal_m_e = df_e_m[df_e_m['emp_ppal'] == 1][PONDERATOR].sum()

        # Guardamos también los "niveles" (conteos ponderados) por si quieres tablas en niveles:
        datos_nacional[f'formal_{etiqueta}_total'] = formal_total_e
        datos_nacional[f'informal_{etiqueta}_total'] = informal_total_e
        datos_nacional[f'formal_{etiqueta}_hombres'] = formal_h_e
        datos_nacional[f'informal_{etiqueta}_hombres'] = informal_h_e
        datos_nacional[f'formal_{etiqueta}_mujeres'] = formal_m_e
        datos_nacional[f'informal_{etiqueta}_mujeres'] = informal_m_e

        # Porcentajes
        datos_nacional[f'pct_formal_{etiqueta}_total'] = safe_pct(formal_total_e, den_total)
        datos_nacional[f'pct_informal_{etiqueta}_total'] = safe_pct(informal_total_e, den_total)

        datos_nacional[f'pct_formal_{etiqueta}_hombres'] = safe_pct(formal_h_e, den_h)
        datos_nacional[f'pct_informal_{etiqueta}_hombres'] = safe_pct(informal_h_e, den_h)

        datos_nacional[f'pct_formal_{etiqueta}_mujeres'] = safe_pct(formal_m_e, den_m)
        datos_nacional[f'pct_informal_{etiqueta}_mujeres'] = safe_pct(informal_m_e, den_m)


    for codigo, etiqueta in niveles_educ.items():
        df_nivel = df_ocupada[df_ocupada['niv_ins'] == codigo]
        datos_nacional[f'ing_{etiqueta}_hombres'] = weighted_average(df_nivel[df_nivel['sex'] == 1], 'ingocup', PONDERATOR)
        datos_nacional[f'ing_{etiqueta}_mujeres'] = weighted_average(df_nivel[df_nivel['sex'] == 2], 'ingocup', PONDERATOR)

    niveles_horas = {
        'primaria': df_ocupada[df_ocupada['niv_ins'].isin([1, 2])],
        'secundaria': df_ocupada[df_ocupada['niv_ins'] == 3],
        'superior': df_ocupada[df_ocupada['niv_ins'] == 4],
    }

    for etiqueta, df_nivel in niveles_horas.items():
        datos_nacional[f'ing_hora_{etiqueta}_hombres'] = weighted_average(df_nivel[df_nivel['sex'] == 1], 'ing_x_hrs', PONDERATOR)
        datos_nacional[f'ing_hora_{etiqueta}_mujeres'] = weighted_average(df_nivel[df_nivel['sex'] == 2], 'ing_x_hrs', PONDERATOR)

    # ------------------------------------------------------------------
    # --- CALCULOS ESTATALES (mismas variables) ---
    # ------------------------------------------------------------------
    datos_estatal = defaultdict(list)

    for ent_code, ent_name in ENTIDADES.items():
        df_base_est = df_base[df_base['ent'] == ent_code]
        df_15_est = df_15_y_mas[df_15_y_mas['ent'] == ent_code]
        df_ocup_est = df_ocupada[df_ocupada['ent'] == ent_code]

        df_15_est_h = df_15_est[df_15_est['sex'] == 1]
        df_15_est_m = df_15_est[df_15_est['sex'] == 2]
        df_ocup_est_h = df_ocup_est[df_ocup_est['sex'] == 1]
        df_ocup_est_m = df_ocup_est[df_ocup_est['sex'] == 2]

        df_ocup_ing_est = df_ocup_est[df_ocup_est['ingocup'] > 0]
        df_ocup_ing_est_h = df_ocup_ing_est[df_ocup_ing_est['sex'] == 1]
        df_ocup_ing_est_m = df_ocup_ing_est[df_ocup_ing_est['sex'] == 2]

        pob_total_est = df_base_est[PONDERATOR].sum()
        pob_15_total_est = df_15_est[PONDERATOR].sum()
        pob_15_h_est = df_15_est_h[PONDERATOR].sum()
        pob_15_m_est = df_15_est_m[PONDERATOR].sum()

        formal_total_est = df_ocup_est[df_ocup_est['emp_ppal'] == 2][PONDERATOR].sum()
        informal_total_est = df_ocup_est[df_ocup_est['emp_ppal'] == 1][PONDERATOR].sum()
        no_remunerados_total_est = df_ocup_est[df_ocup_est['pos_ocu'] == 4][PONDERATOR].sum()

        datos_estatal['year'].append(year)
        datos_estatal['quarter'].append(quarter)
        datos_estatal['ent_code'].append(ent_code)
        datos_estatal['ent_nombre'].append(ent_name)

        datos_estatal['pob_total'].append(pob_total_est)
        datos_estatal['pob_15ymas_total'].append(pob_15_total_est)
        datos_estatal['pob_15ymas_hombres'].append(pob_15_h_est)
        datos_estatal['pob_15ymas_mujeres'].append(pob_15_m_est)
        datos_estatal['pct_15ymas_sobre_total'].append(safe_pct(pob_15_total_est, pob_total_est))

        datos_estatal['pea_total'].append(df_15_est[df_15_est['clase1'] == 1][PONDERATOR].sum())
        datos_estatal['ocupada_total'].append(df_ocup_est[PONDERATOR].sum())
        datos_estatal['desocupada_total'].append(df_15_est[df_15_est['clase2'] == 2][PONDERATOR].sum())

        datos_estatal['masa_salarial_total'].append((df_ocup_ing_est['ingocup'] * df_ocup_ing_est[PONDERATOR]).sum())
        datos_estatal['masa_salarial_hombres'].append((df_ocup_ing_est_h['ingocup'] * df_ocup_ing_est_h[PONDERATOR]).sum())
        datos_estatal['masa_salarial_mujeres'].append((df_ocup_ing_est_m['ingocup'] * df_ocup_ing_est_m[PONDERATOR]).sum())

        datos_estatal['ing_hora_hombres'].append(weighted_average(df_ocup_est_h, 'ing_x_hrs', PONDERATOR))
        datos_estatal['ing_hora_mujeres'].append(weighted_average(df_ocup_est_m, 'ing_x_hrs', PONDERATOR))

        datos_estatal['ing_mensual_hombres'].append(weighted_average(df_ocup_est_h, 'ingocup', PONDERATOR))
        datos_estatal['ing_mensual_mujeres'].append(weighted_average(df_ocup_est_m, 'ingocup', PONDERATOR))

        datos_estatal['horas_sem_hombres'].append(weighted_average(df_ocup_est_h, 'hrsocup', PONDERATOR))
        datos_estatal['horas_sem_mujeres'].append(weighted_average(df_ocup_est_m, 'hrsocup', PONDERATOR))

        datos_estatal['formal_total'].append(formal_total_est)
        datos_estatal['formal_hombres'].append(df_ocup_est_h[df_ocup_est_h['emp_ppal'] == 2][PONDERATOR].sum())
        datos_estatal['formal_mujeres'].append(df_ocup_est_m[df_ocup_est_m['emp_ppal'] == 2][PONDERATOR].sum())

        datos_estatal['informal_total'].append(informal_total_est)
        datos_estatal['informal_hombres'].append(df_ocup_est_h[df_ocup_est_h['emp_ppal'] == 1][PONDERATOR].sum())
        datos_estatal['informal_mujeres'].append(df_ocup_est_m[df_ocup_est_m['emp_ppal'] == 1][PONDERATOR].sum())

        datos_estatal['no_remunerados_total'].append(no_remunerados_total_est)
        datos_estatal['no_remunerados_hombres'].append(df_ocup_est_h[df_ocup_est_h['pos_ocu'] == 4][PONDERATOR].sum())
        datos_estatal['no_remunerados_mujeres'].append(df_ocup_est_m[df_ocup_est_m['pos_ocu'] == 4][PONDERATOR].sum())

        datos_estatal['anios_esc_total'].append(weighted_average(df_15_est, 'anios_esc', PONDERATOR))
        datos_estatal['anios_esc_hombres'].append(weighted_average(df_15_est_h, 'anios_esc', PONDERATOR))
        datos_estatal['anios_esc_mujeres'].append(weighted_average(df_15_est_m, 'anios_esc', PONDERATOR))

        datos_estatal['pob_con_salud'].append(df_ocup_est[df_ocup_est['seg_soc'] == 1][PONDERATOR].sum())
        datos_estatal['pob_sin_salud'].append(df_ocup_est[df_ocup_est['seg_soc'] == 2][PONDERATOR].sum())
        datos_estatal['pct_con_salud'].append(safe_pct(
            datos_estatal['pob_con_salud'][-1],
            datos_estatal['pob_con_salud'][-1] + datos_estatal['pob_sin_salud'][-1]
        ))

        datos_estatal['pct_formal_total_15ymas'].append(safe_pct(formal_total_est, pob_15_total_est))
        datos_estatal['pct_formal_hombres_15ymas'].append(safe_pct(datos_estatal['formal_hombres'][-1], pob_15_h_est))
        datos_estatal['pct_formal_mujeres_15ymas'].append(safe_pct(datos_estatal['formal_mujeres'][-1], pob_15_m_est))

        datos_estatal['pct_informal_total_15ymas'].append(safe_pct(informal_total_est, pob_15_total_est))
        datos_estatal['pct_informal_hombres_15ymas'].append(safe_pct(datos_estatal['informal_hombres'][-1], pob_15_h_est))
        datos_estatal['pct_informal_mujeres_15ymas'].append(safe_pct(datos_estatal['informal_mujeres'][-1], pob_15_m_est))

        datos_estatal['pct_no_remunerados_total_15ymas'].append(safe_pct(no_remunerados_total_est, pob_15_total_est))
        datos_estatal['pct_no_remunerados_hombres_15ymas'].append(safe_pct(datos_estatal['no_remunerados_hombres'][-1], pob_15_h_est))
        datos_estatal['pct_no_remunerados_mujeres_15ymas'].append(safe_pct(datos_estatal['no_remunerados_mujeres'][-1], pob_15_m_est))

        for codigo, etiqueta in niveles_educ.items():
            df_nivel_est = df_ocup_est[df_ocup_est['niv_ins'] == codigo]
            datos_estatal[f'ing_{etiqueta}_hombres'].append(
                weighted_average(df_nivel_est[df_nivel_est['sex'] == 1], 'ingocup', PONDERATOR)
            )
            datos_estatal[f'ing_{etiqueta}_mujeres'].append(
                weighted_average(df_nivel_est[df_nivel_est['sex'] == 2], 'ingocup', PONDERATOR)
            )

        niveles_horas_est = {
            'primaria': df_ocup_est[df_ocup_est['niv_ins'].isin([1, 2])],
            'secundaria': df_ocup_est[df_ocup_est['niv_ins'] == 3],
            'superior': df_ocup_est[df_ocup_est['niv_ins'] == 4],
        }

        for etiqueta, df_nivel_est in niveles_horas_est.items():
            datos_estatal[f'ing_hora_{etiqueta}_hombres'].append(
                weighted_average(df_nivel_est[df_nivel_est['sex'] == 1], 'ing_x_hrs', PONDERATOR)
            )
            datos_estatal[f'ing_hora_{etiqueta}_mujeres'].append(
                weighted_average(df_nivel_est[df_nivel_est['sex'] == 2], 'ing_x_hrs', PONDERATOR)
            )

        datos_estatal['ing_prom_mensual'].append(weighted_average(df_ocup_est, 'ingocup', PONDERATOR))

    df_estatal_trimestre = pd.DataFrame(datos_estatal)
    return pd.Series(datos_nacional), df_estatal_trimestre


# ----------------------------------------------------------------------
# EJECUCION
# ----------------------------------------------------------------------
if __name__ == '__main__':
    periodos = pedir_rango_trimestral()
    resultados_nacionales = []
    resultados_estatales = []

    print(f"\n=== INICIANDO PROCESAMIENTO DE {len(periodos)} TRIMESTRES ===")

    for year, quarter in periodos:
        df_nacional, df_estatal = procesar_trimestre_enoe(year, quarter)

        if not resultados_nacionales and df_nacional is not None:
            estructura_nacional = df_nacional.index
            estructura_estatal = df_estatal.columns

        if df_nacional is not None:
            resultados_nacionales.append(df_nacional)
            resultados_estatales.append(df_estatal)
        else:
            periodo_na = {'year': year, 'quarter': quarter}
            if 'estructura_nacional' in locals():
                serie_na = pd.Series(periodo_na)
                for col in estructura_nacional:
                    if col not in serie_na:
                        serie_na[col] = np.nan
                resultados_nacionales.append(serie_na)

                df_na = pd.DataFrame(periodo_na, index=range(1, 33))
                df_na['ent_code'] = df_na.index
                df_na['ent_nombre'] = df_na['ent_code'].map(ENTIDADES)
                for col in estructura_estatal:
                    if col not in df_na.columns:
                        df_na[col] = np.nan
                resultados_estatales.append(df_na)
                print(f"--- NA para {year} T{quarter} ---")

    if resultados_nacionales:
        df_nac = pd.DataFrame(resultados_nacionales).reset_index(drop=True)
        df_est = pd.concat(resultados_estatales, ignore_index=True)

        today = datetime.now().strftime('%Y%m%d')
        df_nac.to_csv(f"{today}_Nacional.csv", index=False)
        df_est.to_csv(f"{today}_Estados.csv", index=False)

        print("\n=== BASES GUARDADAS CORRECTAMENTE ===")
        print(f"Nacional: {df_nac.shape}")
    else:
        print("\nError: No se generaron datos.")



--- Definicion del Rango de la Serie de Tiempo ---
--- Aviso: Saltando periodo 2020 T1 (No disponible). ---
--- Aviso: Saltando periodo 2020 T2 (No disponible). ---

=== INICIANDO PROCESAMIENTO DE 83 TRIMESTRES ===

--- Procesando: 2005 T1 ---
Cargado: 424,007 regs. Ponderador: FAC

--- Procesando: 2005 T2 ---
Cargado: 428,727 regs. Ponderador: FAC

--- Procesando: 2005 T3 ---
Cargado: 421,751 regs. Ponderador: FAC

--- Procesando: 2005 T4 ---
Cargado: 423,757 regs. Ponderador: FAC

--- Procesando: 2006 T1 ---
Cargado: 426,160 regs. Ponderador: FAC

--- Procesando: 2006 T2 ---
Cargado: 424,579 regs. Ponderador: FAC

--- Procesando: 2006 T3 ---
Cargado: 423,305 regs. Ponderador: FAC

--- Procesando: 2006 T4 ---
Cargado: 421,581 regs. Ponderador: FAC

--- Procesando: 2007 T1 ---
Cargado: 423,910 regs. Ponderador: FAC

--- Procesando: 2007 T2 ---
Cargado: 422,591 regs. Ponderador: FAC

--- Procesando: 2007 T3 ---
Cargado: 418,327 regs. Ponderador: FAC

--- Procesando: 2007 T4 ---
Cargado

C:\Users\rodri\AppData\Local\Temp\ipykernel_23488\2843867770.py:128: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


Cargado: 407,725 regs. Ponderador: FAC

--- Procesando: 2009 T2 ---


C:\Users\rodri\AppData\Local\Temp\ipykernel_23488\2843867770.py:128: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


Cargado: 405,529 regs. Ponderador: FAC

--- Procesando: 2009 T3 ---
Cargado: 402,919 regs. Ponderador: FAC

--- Procesando: 2009 T4 ---
Cargado: 403,862 regs. Ponderador: FAC

--- Procesando: 2010 T1 ---
Cargado: 406,797 regs. Ponderador: FAC

--- Procesando: 2010 T2 ---
Cargado: 408,164 regs. Ponderador: FAC

--- Procesando: 2010 T3 ---
Cargado: 405,533 regs. Ponderador: FAC

--- Procesando: 2010 T4 ---
Cargado: 401,524 regs. Ponderador: FAC

--- Procesando: 2011 T1 ---
Cargado: 402,117 regs. Ponderador: FAC

--- Procesando: 2011 T2 ---
Cargado: 400,977 regs. Ponderador: FAC

--- Procesando: 2011 T3 ---
Cargado: 399,716 regs. Ponderador: FAC

--- Procesando: 2011 T4 ---
Cargado: 399,467 regs. Ponderador: FAC

--- Procesando: 2012 T1 ---
Cargado: 401,880 regs. Ponderador: FAC

--- Procesando: 2012 T2 ---
Cargado: 400,544 regs. Ponderador: FAC

--- Procesando: 2012 T3 ---
Cargado: 397,893 regs. Ponderador: FAC

--- Procesando: 2012 T4 ---
Cargado: 392,632 regs. Ponderador: FAC

--- Proc

C:\Users\rodri\AppData\Local\Temp\ipykernel_23488\2843867770.py:128: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  df = pd.read_stata(file_path, convert_categoricals=False)


Cargado: 392,937 regs. Ponderador: FAC

--- Procesando: 2013 T2 ---
Cargado: 393,107 regs. Ponderador: FAC

--- Procesando: 2013 T3 ---
Cargado: 394,472 regs. Ponderador: FAC

--- Procesando: 2013 T4 ---
Cargado: 400,354 regs. Ponderador: FAC

--- Procesando: 2014 T1 ---
Cargado: 404,014 regs. Ponderador: FAC

--- Procesando: 2014 T2 ---
Cargado: 406,088 regs. Ponderador: FAC

--- Procesando: 2014 T3 ---
Cargado: 405,803 regs. Ponderador: FAC

--- Procesando: 2014 T4 ---
Cargado: 404,640 regs. Ponderador: FAC

--- Procesando: 2015 T1 ---
Cargado: 404,432 regs. Ponderador: FAC

--- Procesando: 2015 T2 ---
Cargado: 403,865 regs. Ponderador: FAC

--- Procesando: 2015 T3 ---
Cargado: 401,825 regs. Ponderador: FAC

--- Procesando: 2015 T4 ---
Cargado: 398,943 regs. Ponderador: FAC

--- Procesando: 2016 T1 ---
Cargado: 397,458 regs. Ponderador: FAC

--- Procesando: 2016 T2 ---
Cargado: 397,156 regs. Ponderador: FAC

--- Procesando: 2016 T3 ---
Cargado: 391,934 regs. Ponderador: FAC

--- Proc